In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [103]:
a = pd.DataFrame(pd.read_csv("stock.txt", sep=','))
a.head()
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14058 entries, 0 to 14057
Data columns (total 7 columns):
Date       14058 non-null object
Open       14058 non-null float64
High       14058 non-null float64
Low        14058 non-null float64
Close      14058 non-null float64
Volume     14058 non-null int64
OpenInt    14058 non-null int64
dtypes: float64(4), int64(2), object(1)
memory usage: 768.9+ KB


In [104]:
b=a.drop("Date", axis=1)
c=b.drop("OpenInt", axis=1)
Stock=c
Stock.head()

,Open,High,Low,Close,Volume
0,0.6277,0.6362,0.6201,0.6201,2575579
1,0.6201,0.6201,0.6122,0.6201,1764749
2,0.6201,0.6201,0.6037,0.6122,2194010
3,0.6122,0.6122,0.5798,0.5957,3255244
4,0.5957,0.5957,0.5716,0.5957,3696430


In [105]:
Stock["Volume"] = Stock["Volume"]/Stock["Volume"].max(axis=0)

In [106]:
from sklearn.model_selection import train_test_split

In [107]:
X = Stock.drop("Close", axis=1)
Y = Stock["Close"] 
X.head()

,Open,High,Low,Volume
0,0.6277,0.6362,0.6201,0.002756
1,0.6201,0.6201,0.6122,0.001888
2,0.6201,0.6201,0.6037,0.002348
3,0.6122,0.6122,0.5798,0.003483
4,0.5957,0.5957,0.5716,0.003955


In [108]:
X = np.array(X)

In [109]:
X.shape

(14058, 4)

In [110]:
Y = np.array(Y) 
Y=Y.reshape(-1,1)

In [111]:
Y.shape 

(14058, 1)

In [112]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)

In [113]:
# bz we want matrix in (nx,m) form so we take transpose
X_train = X_train.T 
X_test = X_test.T
Y_train = Y_train.T 
Y_test = Y_test.T

In [114]:
X_train.shape

(4, 9840)

In [115]:
Y_train.shape

(1, 9840)

In [116]:
# first step is to initialize the parameters
def initialize_parameters(n_a,n_x,n_y): 
    Wf = np.random.randn(n_a, n_a+n_x)
    bf = np.random.randn(n_a,1)
    Wi = np.random.randn(n_a, n_a+n_x)
    bi = np.random.randn(n_a,1)
    Wo = np.random.randn(n_a, n_a+n_x)
    bo = np.random.randn(n_a,1)
    Wc = np.random.randn(n_a, n_a+n_x)
    bc = np.random.randn(n_a,1)
    Wy = np.random.randn(n_y,n_a)
    by = np.random.randn(n_y,1)
    
    parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}
    
    return parameters

In [117]:
# helping function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [118]:
def lstm_cell_forward(xt,a_prev,c_prev,parameters):
#     parameters is dictionary if we want to access Wf then we have to write parameters[Wf], so to avoid writting this 
#     we done this
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    n_x,e = xt.shape
    n_y, n_a = Wy.shape
#     print("lstm_cell_forward")
    concat = np.zeros((n_a+n_x,e))
    concat[: n_a,:] = a_prev
    concat[n_a :,:] = xt
    
    ft = sigmoid(np.dot(Wf,concat) +bf)
    it = sigmoid(np.dot(Wi,concat) +bi)
    cct = np.tanh(np.dot(Wc,concat)+bc)
    c_next = (ft * c_prev) + (it * cct)
    ot = sigmoid(np.dot(Wo, concat)+bo)
    a_next = ot*(np.tanh(c_next))
    
    yt_pred = np.dot(Wy, a_next)+by
    
    cache = (a_next, c_next, a_prev, c_prev, ft,it,cct,ot,xt,parameters)
    
    return a_next, c_next, yt_pred, cache
    

In [119]:
def lstm_forward(x,a0,parameters):
    
    caches = []

    n_x,T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
        
    a = np.zeros((n_a,T_x))
    c = np.zeros((n_a,T_x))
    y = np.zeros((n_y,T_x))
    
#     print("lstm_forward")
        
    a_next = a0
    c_next = np.zeros(a_next.shape)
    
    for t in range(T_x):
        a_next, c_next,yt,cache = lstm_cell_forward(x[:,t].reshape(-1,1),a_next,c_next,parameters)
        a[:,t] = a_next.reshape(-1,)
        c[:,t] = c_next.reshape(-1,)
        y[:,t] = yt.reshape(-1,)
        
        caches.append(cache)
        
    caches = (caches, x)

    return a,y,c,caches

In [120]:
def compute_cost(Y,y):
#     print("compute")
    cost = ((1/2)*(np.power((y-Y),2)))
    YH = (y-Y)
    return cost, YH

In [121]:
def lstm_cell_backward(da_next, dc_next, cache, y_h):
    
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache

    
    n_x,e = xt.shape
    n_a,e = a_next.shape
#     print("stm_cell_backward")
    dot =   da_next*np.tanh(c_next)
    dcct = (da_next*ot*(1-np.power(np.tanh(c_next), 2))+dc_next)*it
    dit =  (da_next*ot*(1-np.power(np.tanh(c_next), 2))+dc_next)*cct
    dft =  (da_next*ot*(1-np.power(np.tanh(c_next), 2))+dc_next)*c_prev
    
    dit =  dit*it*(1-it)
    dft =  dft*ft*(1-ft)
    dot =  dot*ot*(1-ot)
    dcct = dcct*(1-np.power(cct, 2))
    
    concat = np.zeros((n_x + n_a, e))
    concat[: n_a, :] = a_prev
    concat[n_a :, :] = xt
    
    dWf = np.dot(dft, concat.T)
    dWi = np.dot(dit, concat.T)
    dWc = np.dot(dcct, concat.T)
    dWo = np.dot(dot, concat.T)
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True)
    dbc = np.sum(dcct, axis=1, keepdims=True)
    dbo = np.sum(dot, axis=1, keepdims=True)
    
    dWy = np.dot(y_h , a_next.T)
    dby = np.sum(y_h, axis=1, keepdims=True)
    
    
    
    da_prevx = np.dot(parameters['Wf'].T, dft) + np.dot(parameters['Wo'].T, dot) + np.dot(parameters['Wi'].T, dit) + np.dot(parameters['Wc'].T, dcct)  
    da_prev = da_prevx[: n_a, :]
    dc_prev = (da_next*ot*(1-np.power(np.tanh(c_next), 2))+dc_next)*ft
    dxt = da_prevx[n_a :, :]
    
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo, "dWy":dWy,"dby":dby}
    
    return gradients

In [122]:
def lstm_backward(a, caches,YH):
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    

    
    n_a, T_x = a.shape
    n_x,e = x1.shape
    da = np.dot(parameters["Wy"].T , YH)
    dx = np.zeros((n_x, T_x))
    da0 = np.zeros((n_a, e))
    da_prev = np.zeros((n_a, e))
    dc_prev = np.zeros((n_a, e))
    dWf = np.zeros((n_a, n_a + n_x))
    dWi = np.zeros((n_a, n_a + n_x))
    dWc = np.zeros((n_a, n_a + n_x))
    dWo = np.zeros((n_a, n_a + n_x))
    dWy = np.zeros((1, n_a))
    dby = np.zeros((1,e))
    dbf = np.zeros((n_a, 1))
    dbi = np.zeros((n_a, 1))
    dbc = np.zeros((n_a, 1))
    dbo = np.zeros((n_a, 1))
    
    
    for t in reversed(range(T_x)):
        gradients = lstm_cell_backward(da[:,t].reshape(-1,1) + da_prev, dc_prev, caches[t], YH[:,t].reshape(-1,1))
        
        dWf = gradients["dWf"]
        dWi = gradients["dWi"]
        dWc = gradients["dWc"]
        dWo = gradients["dWo"]
        dWy = gradients["dWy"]
        dbf = gradients["dbf"]
        dbi = gradients["dbi"]
        dbc = gradients["dbc"]
        dbo = gradients["dbo"]
        dby = gradients["dby"]
        
        
    da0 = gradients["da_prev"]
    
    gradients = {"dx": dx, "da0": da0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo, "dWy":dWy, "dby":dby}
    return gradients

In [123]:
def update_parameters(parameters, gradients, lr):

    parameters['Wf'] += -lr * gradients['dWf']
    parameters['bf'] += -lr * gradients['dbf']
    parameters['Wi'] += -lr * gradients['dWi']
    parameters['bi'] += -lr * gradients['dbi']
    parameters['Wc'] += -lr * gradients['dWc']
    parameters['bc'] += -lr * gradients['dbc']
    parameters['Wo'] += -lr * gradients['dWo']
    parameters['bo'] += -lr * gradients['dbo']
    parameters['Wy'] += -lr * gradients['dWy']
    parameters['bo'] += -lr * gradients['dbo']
    
    return parameters

In [124]:
def model(X, Y,num_iterations=2000,learning_rate=0.001,print_cost=True):
    costs = []
    n_x,T_x = X.shape
    n_y,T_x = Y.shape
    n_a = 5

    parameters = initialize_parameters(n_a,n_x,n_y)
    a0 = np.random.randn(n_a,1)
    for i in range(0,num_iterations):
        a,y,c,caches = lstm_forward(X,a0, parameters)

        cost,YH = compute_cost(Y,y)
        cost = np.sum(cost)/T_x
        
        gradients = lstm_backward(a,caches,YH)
        a0 +=learning_rate * gradients["da0"]
        parameters = update_parameters(parameters,gradients,learning_rate)
        if print_cost:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost:
            costs.append(cost)
        
#     plt.plot(np.squeeze(costs))
#     plt.ylabel('cost')
#     plt.xlabel('iterations (per tens)')
#     plt.title("Learning rate =" + str(learning_rate))
#     plt.show()
    
    return parameters, a0

In [125]:
parameters,a0 = model(X_train, Y_train)

Cost after iteration 0: 106.299426
Cost after iteration 1: 105.380605
Cost after iteration 2: 104.470951
Cost after iteration 3: 103.570400
Cost after iteration 4: 102.678888
Cost after iteration 5: 101.796352
Cost after iteration 6: 100.922729
Cost after iteration 7: 100.057956
Cost after iteration 8: 99.201970
Cost after iteration 9: 98.354711
Cost after iteration 10: 97.516116
Cost after iteration 11: 96.686124
Cost after iteration 12: 95.864675
Cost after iteration 13: 95.051708
Cost after iteration 14: 94.247164
Cost after iteration 15: 93.450983
Cost after iteration 16: 92.663105
Cost after iteration 17: 91.883472
Cost after iteration 18: 91.112026
Cost after iteration 19: 90.348708
Cost after iteration 20: 89.593460
Cost after iteration 21: 88.846226
Cost after iteration 22: 88.106949
Cost after iteration 23: 87.375571
Cost after iteration 24: 86.652036
Cost after iteration 25: 85.936289
Cost after iteration 26: 85.228274
Cost after iteration 27: 84.527936
Cost after iteration 2

Cost after iteration 231: 43.642336
Cost after iteration 232: 43.734436
Cost after iteration 233: 43.827968
Cost after iteration 234: 43.922919
Cost after iteration 235: 44.019274
Cost after iteration 236: 44.117019
Cost after iteration 237: 44.216141
Cost after iteration 238: 44.316627
Cost after iteration 239: 44.418462
Cost after iteration 240: 44.521633
Cost after iteration 241: 44.626128
Cost after iteration 242: 44.731932
Cost after iteration 243: 44.839033
Cost after iteration 244: 44.947417
Cost after iteration 245: 45.057072
Cost after iteration 246: 45.167985
Cost after iteration 247: 45.280143
Cost after iteration 248: 45.393533
Cost after iteration 249: 45.508142
Cost after iteration 250: 45.623959
Cost after iteration 251: 45.740971
Cost after iteration 252: 45.859165
Cost after iteration 253: 45.978529
Cost after iteration 254: 46.099051
Cost after iteration 255: 46.220719
Cost after iteration 256: 46.343521
Cost after iteration 257: 46.467445
Cost after iteration 258: 46

KeyboardInterrupt: 

In [ ]:
while True:
    features = []
    inp = input(">")
    if inp == 'exit':
        break
        